In [ ]:
%env CUDA_LAUNCH_BLOCKING=1

In [ ]:
from experiments.attn import *

In [ ]:
boards = []
for b in gamegen(8192):
    boards.append(b)
    print(len(boards))
    if len(boards) > 100:
        break
    

In [ ]:
boards = arrdict.cat([arrdict.from_dicts(arrdict.to_dicts(b)) for b in boards])

In [ ]:
import pickle
from pathlib import Path
Path('output/terminal-boards.pkl').wpickle.dumps(boards)

In [ ]:
from rebar import memory
memory.allocations()